# AUTO$^2$ and AUTO-Demos : *tor* - Detection of Torus Bifurcations.

**This is an example on how to use AUTO$^2$ to explore the AUTO Demos *tor* bifurcations and solutions.**

This demo uses a model in Freire, Rodrı́guez-Luis, Gamero & Ponce (1993) to illustrate the
detection of torus bifurcations, branch switching and period-doubling bifurcations.
The equations of the model are given by:

$$
\dot x = (-(\beta+\nu) x +\beta y - a_3 x^3 + b_3 (y-x)^3)/r
$$
$$
\dot y = \beta x - (\beta+\gamma) y - z - b_3 (y-x)^3
$$
$$
\dot z = y
$$

and *it represents a coupling of two elemental circuits: a parallel RCL-circuit and a RC-circuit*.

The free parameter in the current example notebook is $\nu$ . Other parameters are fixed.

We are thus going to find the fixed points and periodic orbits of this system and continue them by varying $\nu$.

#### References

* Freire, E., Rodriguez-Luis, A. J., Gamero, E., & Ponce, E. (1993). A case study for homoclinic chaos in an autonomous electronic circuit: A trip from Takens-Bogdanov to Hopf-Šil'nikov. Physica D: Nonlinear Phenomena, 62(1-4), 230-253. [doi:10.1016/0167-2789(93)90284-8](https://doi.org/10.1016/0167-2789(93)90284-8)

## Code

First we set the Python path if needed:

In [ ]:
import sys, os

In [ ]:
sys.path.extend([os.path.abspath('../../../')])

And load the needed libraries, including AUTO$^2$:

In [ ]:
import numpy as np
from numba import njit
from scipy.optimize import root
from scipy.integrate import solve_ivp

In [ ]:
from auto2.diagrams.bifurcations import BifurcationDiagram

Creating the model equations

In [ ]:
@njit
def tor(X, nu, beta, gamma, r, a3, b3):
    x = X[0]
    y = X[1]
    z = X[2]
    F = np.zeros(3)
    F[0]= ( -(beta+nu)*x + beta*y - a3*x**3 + b3*(y-x)**3 )/r
    F[1]=  beta*x - (beta+gamma)*y - z - b3*(y-x)**3
    F[2]= y

    return F

@njit
def tort(t, X, nu, beta, gamma, r, a3, b3):
    return tor(X, nu, beta, gamma, r, a3, b3)

and define a set of standard parameters:

In [ ]:
params = {
    'nu': -0.9,
    'beta': 0.5,
    'gamma': -0.6,
    'r': 0.6,
    'a3': 0.328578,
    'b3': 0.933578,
}

For reference later, we can compute a long trajectory on the attractor of this model for $\nu=-0.61$ :

In [ ]:
par = params.copy()
par['nu'] = -0.61

In [ ]:
parv = tuple([par['nu'], par['beta'], par['gamma'], par['r'], par['a3'], par['b3']])

In [ ]:
#first a transient
# ic = np.zeros(3) + 0.0001
ic = np.array([ 2.67030e-01  , 2.22339e-26  , 1.51291e-01])
transient = solve_ivp(tort, (0., 1000.), ic, args=parv)

In [ ]:
# then the trajectory itself
ic = transient['y'][:, -1]
trajectory = solve_ivp(tort, (0., 10000.), ic, args=parv)

Next we setup the list of points and parameters that AUTO$^2$ will have to continue. We are providing only one point for the value $\nu = -0.9$ :

In [ ]:
initial_points = list()

initial_points.append({'parameters': params, 'initial_data': np.array([0., 0., 0.])})

We are now ready to compute the diagram of fixed points as a function of $\nu$. Note that we specify that the bifurcation diagram object must load the ̀`tor.f90` and `c.tor` files where the model equations and continuation parameters have been written.

In [ ]:
b = BifurcationDiagram('tor')

b.compute_fixed_points_diagram(initial_points,extra_comparison_parameters=['x', 'y'], comparison_tol=[1.e-3] * 3,
                               ICP=['nu'], NMX=300, UZSTOP={'nu':[-0.91,-0.4]}, UZR={'nu': list(np.arange(-0.8, -0.4, 0.1))+[par['nu'],]}, NPR=0)

We can now plot the result as functions of $\nu$ and $L^2$ norm :

In [ ]:
b.plot_fixed_points_diagram();

and also as functions of $x$ and $z$ :

In [ ]:
b.plot_fixed_points_diagram((2,4));

or in 3D as functions of $\nu$, $L^2$ norm and $x$ :

In [ ]:
b.plot_fixed_points_diagram_3D();

We see that 2 branches were found. The second one presents 2 distinct Hopf bifurcations.

We can continue periodic orbits out of these Hopf bifurcations : 

In [ ]:
b.compute_periodic_orbits_diagram(
    5, 
    extra_comparison_parameters=['x', 'y'], 
    max_number_bp=None, 
    comparison_tol=[1.e-4, 1.e-4, 1.e-4],
    ICP=['nu']
)


and plot the results on a bifurcation diagram:

In [ ]:
ax = b.plot_fixed_points_diagram()
b.plot_periodic_orbits_diagram(ax=ax, cmap='gist_ncar');

We can also plot both the bifurcation diagram and the solutions for a given value of $\rho$:

In [ ]:
b.plot_diagram_and_solutions(-0.8, solutions_variables=(0, 1), fixed_points_diagram_kwargs={'legend': True}, 
                             periodic_orbits_diagram_kwargs={'cmap': 'gist_ncar'});

which shows symmetrically connected fixed point via a heteroclinic connection, and possibly a close Shilnikov bifurcation.

Plotting a single branch and its solutions:

In [ ]:
# plotting branch 4
b.plot_single_po_branch_and_solutions(4, cmap='Blues_r')

shows an homoclinic connection for smaller values of the parameter $\nu$.

You can compare these results with the animated figure at the top of the notebook.
We can also plot the result in 3 dimensions to get a better view:

In [ ]:
b.plot_diagram_in_3D_and_solutions_in_3D(-0.8, solutions_variables=(0, 1, 2), fixed_points_diagram_kwargs={}, 
                                         periodic_orbits_diagram_kwargs={'cmap': 'gist_ncar'});

Finally, it is not hard to also plot the dynamics on the attractor (represented by the long trajectory computed beforehand) on top of the solutions to see their relevance:

In [ ]:
axs = b.plot_diagram_and_solutions(par['nu'], solutions_variables=(0, 1), fixed_points_diagram_kwargs={'legend': True}, 
                             periodic_orbits_diagram_kwargs={'cmap': 'gist_ncar'})
axs[1].plot(trajectory['y'][0], trajectory['y'][1], marker='o', ms=0.37, ls='', color='darkgray');

but as you can see, the found bifurcations (torus, homoclinic, heteroclinic, ...) do not really relate to the actual dynamics. For a more interesting case, please look at the `tor-shil` derivated demo.